In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

from huggingface_hub import hf_hub_download
from PIL import Image

#### Load dataset

In [ ]:
import boto3
import config
from dataloader import SNV3Dataset, collate_fn
from torch.utils.data import Dataset, DataLoader
import time

session = boto3.Session(
    aws_access_key_id=config.aws_access_key_id,
    aws_secret_access_key=config.aws_secret_access_key,
)
s3 = session.resource('s3')
train_ds = SNV3Dataset(split='train',
                  bucket='soccernet-230',
                  vocab_path='vocab_files/train_vocab.pyi',
                  num_clips=1,
                  include_vid=True,
                     local_video_path="/raid/videos")
train_dataloader = DataLoader(train_ds, collate_fn=collate_fn, batch_size=64, num_workers=4)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(k,v.shape)
     

#### Define Model

In [1]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

from huggingface_hub import hf_hub_download
from PIL import Image

# video captioning
processor = AutoProcessor.from_pretrained("microsoft/git-base-vatex")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-vatex")

NameError: name 'AutoProcessor' is not defined

#### Dummy forward pass

In [ ]:
outputs = model(input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                pixel_values=batch["pixel_values"],
                labels=batch["input_ids"])
outputs.loss

#### Train the model

In [ ]:


import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()

for epoch in range(50):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)
    
    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()